In [1]:
#librairies :
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib
import tkinter as tk
from scipy.signal import butter, lfilter
from scipy import signal
import matplotlib.image as image
from tkinter import scrolledtext
import time
import tkinter.filedialog
import os

In [2]:
def choose_video():
    filename = tk.filedialog.askopenfilename()
    cap = cv2.VideoCapture(filename)
    return cap


def coup() :
    resultat=[]
    resultats=[]
    cap=choose_video()
    while True:
        ret, frame = cap.read() # lire le frame suivant
        if not ret:
                break
                
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # convertir en niveaux de gris
        frame = np.array(frame, dtype=np.int32) #convertir en matrice
        
        lignemilieu=frame[420,:]

        result=sum(lignemilieu)
        
        resultat.append(result)

    moy=np.sum(resultat)/len(resultat)
    resultat=  [(i/moy)-1 for i in resultat]
    resultats=np.array(resultat)
       
            
    return resultats
    


#video=choose_video()

def getFrame(seconde): 
    video=choose_video()
    video.set(cv2.CAP_PROP_POS_MSEC,seconde*1000) #renvoie le timestamp de chaque image
    hasFrames,image = video.read() #si le frame est lu correctement, donc si la video est pas finie, hasFrames renvoie true
    if hasFrames:
        cv2.imwrite("imagen"+str(count)+".jpg", image)
    return hasFrames

seconde = 0
frameRate = 0.03 #//it will capture image in each 0.1 second
count=1
success = getFrame(seconde) 
while success:
    count = count + 1
    seconde = seconde + frameRate
    seconde = round(seconde, 2)
    success = getFrame(seconde)
    


def coupe(): #normalisation
    resultat=[]
    #video=choose_video()
    for i in range(1, count):
        img=image.imread("imagen"+str(i)+".jpg") #je converti mes images en matrice
        lignemilieu=img[420,:,:]
        result = [sum(l) for l in zip(*lignemilieu)]
        result= sum(result)
        resultat.append(result)
    
        i=i+1
   
    moy=np.sum(resultat)/len(resultat)
    resultat=  [(i/moy)-1 for i in resultat]
    resultats=np.array(resultat)
    
    return resultats

In [3]:
def passebande(resultats):
    # Fréquence d'échantillonnage (Hz)
    fs = 30

    # Coefficients de Butterworth pour le filtre passe-bande
    low = 0.67 / (0.5 * fs)  #0.3 car ca correspond a un bpm max de 180
    high = 3 / (0.5 * fs) #bpm de 40
    b, a = signal.butter(4, [low, high], btype='band')

    # Appliquer le filtre passe-bande
    signal_out = signal.filtfilt(b, a, resultats)

    # Affichage du signal filtré
    #print(signal_out)

    #ft = signal.TransferFunction(b, [1])
    return signal_out

In [4]:
def freqcardiaque(signalfiltre):
    fs = 30
    tf = np.abs((np.fft.fftshift(np.fft.fft(signalfiltre))))
    f = np.linspace(-fs/2, fs/2, len(tf)) 


    # Affichage de la TF en fonction de la fréquence
    #plt.plot(frequencies, tf)
    # Affichage du graphique
    #plt.plot(f, tf)
    #plt.xlim([0,5])
    #plt.xlabel("Fréquence (Hz)")
    #plt.ylabel("Amplitude de la TF")
    #plt.show()


    #Trouver l'indice de la fréquence cardiaque (fréquence la plus élevée dans la bande de fréquences cardiaques)
   
    x=np.argmax(tf)
    #print(x)
    fc1 = np.abs(f[x])
    fc=(fc1)*60
    #print('BPM = ', fc)
    return fc
    


In [5]:
def fc():
    r=coup()
    signalfiltre= passebande(r)
    fc= freqcardiaque(signalfiltre)
    root.destroy
    return fc


#interface :
def button1_function():
    print("Mesure de la saturation O2")

def button2_function():
    print("Mesure de la frequence cardiaque ")

root = tk.Tk()
root.geometry("500x500+500+50")
root.configure(bg='light blue')
welcome_label = tk.Label(root, text="Bienvenue sur PolyOxy!", font=("Arial", 24), bg='light blue')
welcome_label.pack()


button1 = tk.Button(root, text="SPO2",font=("Arial", 12), bg='dark blue', command=button1_function)
button1.place(x=200, y=200)
button1.config(padx=100, pady=30)
button1.pack()

button2 = tk.Button(root, text="BPM",font=("Arial", 12),bg='dark blue', command=fc)
button2.place(x=300, y=200)
button2.config(padx=100, pady=30)
button2.pack()

root.mainloop()

pb encore: il faut re executer getframe a chaque fois avant d'executer le code, donc c'est bien a getframe que je choisi ma video


In [9]:
###############################calibration pour spo2##########################

def calibration():
    dossier = 'C:/Users/leael/OneDrive/Desktop/sat'
    #video_files = [f for f in os.listdir(directory) if f.endswith('.mp4')]
    nomvideos = []
    valmoys = []

    for nomvideo in os.listdir(dossier):
        # Lire la vidéo
        video = cv2.VideoCapture(os.path.join(dossier, nomvideo))

        # Initialiser la somme des pixels pour toutes les frames
        sommepix = 0

        # Initialiser le compteur de frames
        frame_count = 0

        # Boucle sur toutes les frames de la vidéo
        while True:
            ret, frame = video.read()
            if not ret:
                break
            frame_count += 1
            sommepix += np.sum(frame)

        # Fermer la vidéo
        video.release()

        # Calculer la moyenne des pixels pour toutes les frames de la vidéo
        valmoy = sommepix / frame_count

        # Ajouter le nom de la vidéo et la valeur moyenne à la liste
        nomvideos.append(nomvideo)
        valmoys.append(valmoy)

    # Afficher le graphique
    plt.scatter(nomvideos, valmoys)
    plt.xlabel('Vidéos')
    plt.ylabel('Moyenne des pixels')
    plt.show()

    #minn=np.min(valmoys)
    #maxx=np.max(valmoys)
    #print('intervalle:', minn, maxx)
    
    num=[]
    for i in nomvideos :
        prefix = i[:2]
        # Convertir les lettres en nombre
        number = int(prefix)
        num.append(number)

    
    coefficients = np.polyfit(num, valmoys, 1)

    # Afficher l'équation de régression (y = mx + b)
    m = coefficients[0]
    b = coefficients[1]
    print("Equation de regression: y = {}x + {}".format(m, b))
    
    return m, b


def calcul(m,b) :
    resultats=[]
    cap=choose_video()
    #cap=cv2.VideoCapture("C:/Users/leael/Dropbox/PC/Downloads/fcastrid.mp4")
        
        
        # Initialiser la somme des pixels pour toutes les frames
    sommepix = 0

    # Initialiser le compteur de frames
    frame_count = 0

    # Boucle sur toutes les frames de la vidéo
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        sommepix += np.sum(frame)
    
    # Fermer la vidéo
    cap.release()

    # Calculer la moyenne des pixels pour toutes les frames de la vidéo
    valmoy = sommepix / frame_count
    
    o2=(valmoy-b)/m
        
    return o2

In [10]:
def spo2():
    m,b=calibration()
    spo2=calcul(m,b)
    
    return spo2
    


In [21]:
#interface :
    
def open_second_window():
    # Create the second window
    second_window = tk.Tk()
    second_window.configure(bg='')
    second_window.title("Resultat :")
    #second_window.geometry("200x200+5+0")
    screen_width = second_window.winfo_screenwidth()
    screen_height = second_window.winfo_screenheight()

    # Calculate the x and y coordinates for the tkinter window
    x = int((screen_width - second_window.winfo_reqwidth()) / 2)
    y = int((screen_height - second_window.winfo_reqheight()) / 2)

    # Set the position of the tkinter window to the center of the screen
    second_window.geometry("+{}+{}".format(x, y))
    
    
    fc2=fc()
    text_label2 = tk.Label(second_window, text='Le BPM est de :', font=("Arial", 12))
    text_label = tk.Label(second_window, text=fc2, font=("Arial", 12))
    text_label2.pack()
    text_label.pack()
    
    root.destroy()
    
def ow():
    # Create the second window
    third_window = tk.Tk()
    third_window.title("Resultat :")
    #third_window.geometry("200x200+5+0")
    
    screen_width = third_window.winfo_screenwidth()
    screen_height = third_window.winfo_screenheight()

    # Calculate the x and y coordinates for the tkinter window
    x = int((screen_width - third_window.winfo_reqwidth()) / 2)
    y = int((screen_height - third_window.winfo_reqheight()) / 2)

    # Set the position of the tkinter window to the center of the screen
    third_window.geometry("+{}+{}".format(x, y))
    
    
    
    
    
    spo22=spo2()
    text_label3 = tk.Label(third_window, text='Le taux dO2 est de :', font=("Arial", 12))
    text_label = tk.Label(third_window, text=spo22, font=("Arial", 12))
    text_label3.pack()
    text_label.pack()


    # Destroy the first window
    root.destroy()


root = tk.Tk()
root.geometry("1000x600+0+0")
root.configure(bg='light blue')


#bien mettre le chemin de l'image
bg = tk.PhotoImage(file = "C:/Users/leael/Dropbox/PC/Downloads/rsz_1anatoheart-overlay.png")

canvas1 = tk.Canvas(root, width = 1036, height = 600)
canvas1.pack(fill = "both", expand = True)


canvas1.create_image( 0, 0, image = bg,anchor = "nw")
canvas1.create_text( 260, 150, text = " o l y  O x y", font=("Times New Roman",30,"bold"), fill='grey')
#r=canvas1.create_rectangle(canvas1.bbox(i),fill="grey")
#canvas1.tag_lower(r,i)


#ca marche ici, donc faire meme chose pour SPO2
button1 = tk.Button(root, text="BPM",font=("Arial", 12), bg="grey",fg="black", command=open_second_window)
button1.place(x=203, y=250)
button1.config(padx=20, pady=10)

button2 = tk.Button(root, text="SPO2",font=("Arial", 12),bg="grey",fg="black", command=ow)
button2.place(x=200, y=310)
button2.config(padx=20, pady=10)


menubar = tk.Menu(root)
filemenu = tk.Menu(menubar, tearoff=0)
filemenu.add_command(label="Quitter", command=root.destroy)
menubar.add_cascade(label="Fichier", menu=filemenu)
root.config(menu=menubar)

# create a Text widget with a Scrollbar attached
txt = scrolledtext.ScrolledText(root, undo=True)
txt['font'] = ('consolas', '12')
txt.pack(expand=True, fill='both')

root.mainloop()# create a menu bar with an Exit command

BPM =  71.62629757785474
